In [2]:
import jax
import jax.numpy as jnp
import numpy as np

/Users/qwe/miniforge3/lib/python3.8/site-packages/jax/_src/lib/__init__.py:32: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


In [24]:
np.random.seed(1)
N = 5
beta = 1
choices = np.random.rand(N)

In [27]:
sigma_z = jnp.array([[1,0],[0,-1]])
Pr = (jnp.identity(2) - sigma_z)/2
eps = jnp.array(0.5 * np.random.rand(N))

rho_prod = (1/(1 + jnp.exp(- beta*eps[0]))) * jnp.expm1(- beta* eps[0] * Pr)

def rho_prod_sacn(carry,eps_r):
    y = (1/(1 + jnp.exp(- beta*eps_r))) * jnp.expm1(- beta* eps_r * Pr)
    return jnp.kron(carry,y),0



for i in range(1,N):
    eps_r = eps[i]
    y = (1/(1 + jnp.exp(- beta*eps_r))) * jnp.expm1(- beta* eps_r * Pr)
    rho_prod = jnp.kron(rho_prod,y)

#jax.lax.scan(rho_prod_sacn,init,eps[1:])
y



DeviceArray([[-0.        , -0.        ],
             [-0.        , -0.00684679]], dtype=float32)

In [26]:
jnp.sum(jnp.abs(rho_prod)>0)

DeviceArray(1, dtype=int32)

In [33]:
np.outer(x,x.T) @ x

array([0.30151134, 0.30151134, 0.        , 0.        , 0.30151134,
       0.30151134, 0.30151134, 0.30151134, 0.30151134, 0.        ,
       0.        , 0.30151134, 0.        , 0.30151134, 0.30151134,
       0.        , 0.        , 0.30151134, 0.        , 0.        ])

In [ ]:
np.random( )